[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rcrowe-google/Learning-JAX/blob/main/code-exercises/01%20-%20JAX%20AI%20Stack.ipynb)

# Introduction

**Welcome to the JAX AI Stack Exercises!**

This notebook is designed to accompany the "Leveraging the JAX AI Stack" lecture.  You'll get hands-on experience with core JAX concepts, Flax NNX for model building, Optax for optimization, and Orbax for checkpointing.

The exercises will guide you through implementing key components, drawing parallels to PyTorch where appropriate, to solidify your understanding.

Let's get started!

In [1]:
# @title Setup: Install and Import Libraries
# Install necessary libraries
# !pip install -q jax-ai-stack==2025.9.3

import jax
import jax.numpy as jnp
import flax
from flax import nnx
import optax
import orbax.checkpoint as ocp # For Orbax
from typing import Any, Dict, Tuple # For type hints

# Helper to print PyTrees more nicely for demonstration
import pprint
import os # For Orbax directory management
import shutil # For cleaning up Orbax directory

print(f"JAX version: {jax.__version__}")
print(f"Flax version: {flax.__version__}")
print(f"Optax version: {optax.__version__}")
print(f"Orbax version: {ocp.__version__}")

# Global JAX PRNG key for reproducibility in exercises
# Students can learn to split this key for different operations.
main_key = jax.random.key(0)

JAX version: 0.8.0
Flax version: 0.12.0
Optax version: 0.2.6
Orbax version: 0.11.26


### Exploring schmap'ing

In [2]:
from jax.sharding import Mesh, PartitionSpec as P

n_cuda_devices = 4

mesh = jax.make_mesh((n_cuda_devices,), ('x',))

@jax.jit
def f_elementwise(x):
    return 2 * jnp.sin(x) + 1

f_elementwise_sharded = jax.shard_map(
    f_elementwise,
    mesh=mesh,
    in_specs=P('x'),
    out_specs=P('x')
)    

f_elementwise_sharded

arr = jnp.arange(32)

sharded = f_elementwise_sharded(arr)
sharded

Array([ 1.        ,  2.682942  ,  2.818595  ,  1.28224   , -0.513605  ,
       -0.9178486 ,  0.44116902,  2.3139732 ,  2.9787164 ,  1.824237  ,
       -0.08804214, -0.99998045, -0.07314587,  1.8403342 ,  2.9812148 ,
        2.3005757 ,  0.42419332, -0.92279494, -0.50197446,  1.2997544 ,
        2.8258905 ,  2.6733112 ,  0.98229736, -0.69244087, -0.81115675,
        0.7352965 ,  2.525117  ,  2.912752  ,  1.5418116 , -0.32726777,
       -0.97606325,  0.19192469], dtype=float32)

In [3]:
jax.debug.visualize_array_sharding(sharded)

┌───────┬───────┬───────┬───────┐
│ GPU 0 │ GPU 1 │ GPU 2 │ GPU 3 │
└───────┴───────┴───────┴───────┘

## Exercise 1: JAX Core & NumPy API

**Goal**: Get familiar with jax.numpy and JAX's functional programming style.

### Instructions:

1. Create two JAX arrays, a (a 2x2 matrix of random numbers) and b (a 2x2 matrix of ones) using jax.numpy (jnp). You'll need a jax.random.key for creating random numbers.
2. Perform element-wise addition of a and b.
3. Perform matrix multiplication of a and b.
4. Demonstrate JAX's immutability:
 - Store the Python id() of array a.
 - Perform an operation like a = a + 1.
 - Print the new id() of a and observe that it has changed, indicating a new array was created.

In [4]:
# Instructions for Exercise 1
key_ex1, main_key = jax.random.split(main_key) # Split the main key

# 1. Create JAX arrays a and b
# Create array 'a' (2x2 random normal) and 'b' (2x2 ones)
a = jax.random.normal(key_ex1, (2,2))
b = jnp.ones((2,2))

print("Array a:\n", a)
print("Array b:\n", b)

# 2. Perform element-wise addition
# Add a and b
c = a + b 
print("Element-wise sum c = a + b:\n", c)

# 3. Perform matrix multiplication
# Matrix multiply a and b
d = jnp.matmul(a, b) # or a @ b 
print("Matrix product d = a @ b:\n", d)

# 4. Demonstrate immutability
original_a_id = id(a)
print(f"Original id(a): {original_a_id}")

# Perform an operation that reassigns 'a', e.g., a = a + 1
a_new_ref = a + 1 
new_a_id = id(a_new_ref)
print(f"New id(a) after 'a = a + 1': {new_a_id}")

# Check if original_a_id is different from new_a_id
print(f"IDs are different: {original_a_id != new_a_id}")

Array a:
 [[ 1.0040143 -0.9063372]
 [-0.7481722 -1.1713669]]
Array b:
 [[1. 1.]
 [1. 1.]]
Element-wise sum c = a + b:
 [[ 2.0040143   0.0936628 ]
 [ 0.25182778 -0.17136693]]
Matrix product d = a @ b:
 [[ 0.09767705  0.09767705]
 [-1.9195392  -1.9195392 ]]
Original id(a): 103054992
New id(a) after 'a = a + 1': 109075296
IDs are different: True


In [5]:
# @title Solution 1: JAX Core & NumPy API
key_ex1_sol, main_key = jax.random.split(main_key)

# 1. Create JAX arrays a and b
a_sol = jax.random.normal(key_ex1_sol, (2, 2))
b_sol = jnp.ones((2, 2))

print("Array a:\n", a_sol)
print("Array b:\n", b_sol)

# 2. Perform element-wise addition
c_sol = a_sol + b_sol
print("Element-wise sum c = a + b:\n", c_sol)

# 3. Perform matrix multiplication
d_sol = jnp.dot(a_sol, b_sol) # or d = a @ b
print("Matrix product d = a @ b:\n", d_sol)

# 4. Demonstrate immutability
original_a_id_sol = id(a_sol)
print(f"Original id(a_sol): {original_a_id_sol}")

a_sol_new_ref = a_sol + 1 # This creates a new array and rebinds the Python variable.
new_a_id_sol = id(a_sol_new_ref)
print(f"New id(a_sol_new_ref) after 'a_sol = a_sol + 1': {new_a_id_sol}")
print(f"IDs are different: {original_a_id_sol != new_a_id_sol}")
print("This shows that the original array was not modified in-place; a new array was created.")

Array a:
 [[ 0.19890743 -0.77017707]
 [-1.2587329  -0.4105852 ]]
Array b:
 [[1. 1.]
 [1. 1.]]
Element-wise sum c = a + b:
 [[ 1.1989074   0.22982293]
 [-0.25873291  0.58941483]]
Matrix product d = a @ b:
 [[-0.57126963 -0.57126963]
 [-1.6693181  -1.6693181 ]]
Original id(a_sol): 96016912
New id(a_sol_new_ref) after 'a_sol = a_sol + 1': 110526112
IDs are different: True
This shows that the original array was not modified in-place; a new array was created.


## Exercise 2: jax.jit (Just-In-Time Compilation)

**Goal**: Understand how to use jax.jit to compile JAX functions for performance.

### Instructions:

1. Define a Python function compute_heavy_stuff(x, w, b) that performs a sequence of jnp operations:
 - y = jnp.dot(x, w)
 - y = y + b
 - y = jnp.tanh(y)
 - result = jnp.sum(y)
 - Return result.
2. Create a JIT-compiled version of this function, fast_compute_heavy_stuff, using jax.jit.
3. Create some large dummy JAX arrays for x, w, and b.
4. Call both the original and JIT-compiled functions with the dummy data.
5. (Optional) Use the `%timeit` magic command in Colab (in separate cells) to compare their execution speeds. Remember that the first call to a JIT-compiled function includes compilation time.

In [6]:
# Instructions for Exercise 2
key_ex2_main, main_key = jax.random.split(main_key)
key_ex2_x, key_ex2_w, key_ex2_b = jax.random.split(key_ex2_main, 3)

# 1. Define the Python function
def compute_heavy_stuff(x, w, b):
    y = jnp.dot(x, w)
    y = y + b
    y = jnp.tanh(y)
    result = jnp.sum(y)
    return result

# 2. Create a JIT-compiled version
fast_compute_heavy_stuff = jax.jit(compute_heavy_stuff)

# 3. Create dummy data
dim1, dim2, dim3 = 500, 1000, 500
x_data = jax.random.normal(key_ex2_x, (dim1, dim2))
w_data = jax.random.normal(key_ex2_w, (dim2, dim3))
b_data = jax.random.normal(key_ex2_b, (dim3,))

# 4. Call both functions
result_original = compute_heavy_stuff(x_data, w_data, b_data).block_until_ready()
result_fast_first_call = fast_compute_heavy_stuff(x_data, w_data, b_data).block_until_ready() # First call (compiles)
result_fast_second_call = fast_compute_heavy_stuff(x_data, w_data, b_data).block_until_ready() # Second call (uses compiled)

print(f"Result (original): {result_original}")
print(f"Result (fast, 1st call): {result_fast_first_call}")
print(f"Result (fast, 2nd call): {result_fast_second_call}")

if result_original is not None and result_fast_first_call is not None:
  assert jnp.allclose(result_original, result_fast_first_call), "Results should match!"
  print("\nResults from original and JIT-compiled functions match.")

# 5. Optional: Timing (use %timeit in separate cells for accuracy)
print("\nTo see the speed difference, run these in separate cells:")



Result (original): -56.246559143066406
Result (fast, 1st call): -56.246559143066406
Result (fast, 2nd call): -56.246559143066406

Results from original and JIT-compiled functions match.

To see the speed difference, run these in separate cells:


In [57]:
print("%timeit compute_heavy_stuff(x_data, w_data, b_data).block_until_ready()")
%timeit compute_heavy_stuff(x_data, w_data, b_data).block_until_ready()

%timeit compute_heavy_stuff(x_data, w_data, b_data).block_until_ready()
126 μs ± 6.94 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [58]:
print("%timeit fast_compute_heavy_stuff(x_data, w_data, b_data).block_until_ready()")
%timeit fast_compute_heavy_stuff(x_data, w_data, b_data).block_until_ready()

%timeit fast_compute_heavy_stuff(x_data, w_data, b_data).block_until_ready()
51.4 μs ± 118 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Exercise 3: jax.grad (Automatic Differentiation)

**Goal**: Learn to use jax.grad to compute gradients of functions.

### Instructions:

1. Define a Python function scalar_loss(params, x, y_true) that:
 - Takes a dictionary params with keys 'w' and 'b'.
 - Computes y_pred = params['w'] * x + params['b'].
 - Returns a scalar loss, e.g., jnp.mean((y_pred - y_true)**2).
2. Use jax.grad to create a new function, compute_gradients, that computes the gradient of scalar_loss with respect to its first argument (params).
3. Initialize some dummy params, x_input, and y_target values.
4. Call compute_gradients to get the gradients. Print the gradients.

In [ ]:
# Instructions for Exercise 3

# 1. Define the scalar_loss function
def scalar_loss(params: Dict[str, jnp.ndarray], x: jnp.ndarray, y_true: jnp.ndarray) -> jnp.ndarray:
    # DONE: Implement the prediction and loss calculation
    y_pred = params["w"] * x + params["b"]
    loss = jnp.mean((y_true - y_pred)**2)
    return loss

# 2. Create the gradient function using jax.grad
# DONE: Gradient of scalar_loss w.r.t. 'params' (argnums=0)
compute_gradients = jax.grad(scalar_loss, argnums=0)

# 3. Initialize dummy data
params_init = {'w': jnp.array(2.0), 'b': jnp.array(1.0)}
x_input_data = jnp.array([1.0, 2.0, 3.0])
y_target_data = jnp.array([7.0, 9.0, 11.0]) # Targets for y = 3x + 4 (to make non-zero loss with init_params)

# 4. Call the gradient function
gradients = compute_gradients(params_init, x_input_data, y_target_data) 
print("Initial params:", params_init)
print("Gradients w.r.t params:\n", gradients)

# Expected gradients (manual calculation for y_pred = wx+b, loss = mean((y_pred - y_true)^2)):
# dL/dw = mean(2 * (wx+b - y_true) * x)
# dL/db = mean(2 * (wx+b - y_true) * 1)
# For params_init={'w': 2.0, 'b': 1.0}, x=[1,2,3], y_true=[7,9,11]
# x=1: y_pred = 2*1+1 = 3. Error = 3-7 = -4. dL/dw_i_term = 2*(-4)*1 = -8.  dL/db_i_term = 2*(-4)*1 = -8
# x=2: y_pred = 2*2+1 = 5. Error = 5-9 = -4. dL/dw_i_term = 2*(-4)*2 = -16. dL/db_i_term = 2*(-4)*1 = -8
# x=3: y_pred = 2*3+1 = 7. Error = 7-11 = -4. dL/dw_i_term = 2*(-4)*3 = -24. dL/db_i_term = 2*(-4)*1 = -8
# Mean gradients: dL/dw = (-8-16-24)/3 = -48/3 = -16.  dL/db = (-8-8-8)/3 = -24/3 = -8.
if gradients is not None:
    assert jnp.isclose(gradients['w'], -16.0)
    assert jnp.isclose(gradients['b'], -8.0)
    print("\nGradients match expected values.")

Initial params: {'w': Array(2., dtype=float32, weak_type=True), 'b': Array(1., dtype=float32, weak_type=True)}
Gradients w.r.t params:
 {'b': Array(-8., dtype=float32, weak_type=True), 'w': Array(-16., dtype=float32, weak_type=True)}

Gradients match expected values.


In [ ]:
# @title Solution 3: `jax.grad` (Automatic Differentiation)

# 1. Define the scalar_loss function
def scalar_loss_sol(params: Dict[str, jnp.ndarray], x: jnp.ndarray, y_true: jnp.ndarray) -> jnp.ndarray:
    y_pred = params['w'] * x + params['b']
    loss = jnp.mean((y_pred - y_true)**2)
    return loss

# 2. Create the gradient function using jax.grad
# Gradient of scalar_loss w.r.t. 'params' (which is the 0-th argument)
compute_gradients_sol = jax.grad(scalar_loss_sol, argnums=0)

# 3. Initialize dummy data
params_init_sol = {'w': jnp.array(2.0), 'b': jnp.array(1.0)}
x_input_data_sol = jnp.array([1.0, 2.0, 3.0])
y_target_data_sol = jnp.array([7.0, 9.0, 11.0])

# 4. Call the gradient function
gradients_sol = compute_gradients_sol(params_init_sol, x_input_data_sol, y_target_data_sol)
print("Initial params:", params_init_sol)
print("Gradients w.r.t params:\n", pprint.pformat(gradients_sol))

# Verify with expected values (calculated in instructions)
expected_dL_dw = -16.0
expected_dL_db = -8.0
assert jnp.isclose(gradients_sol['w'], expected_dL_dw), f"Grad w.r.t 'w' is {gradients_sol['w']}, expected {expected_dL_dw}"
assert jnp.isclose(gradients_sol['b'], expected_dL_db), f"Grad w.r.t 'b' is {gradients_sol['b']}, expected {expected_dL_db}"
print("\nGradients match expected values.")

## Exercise 4: jax.vmap (Automatic Vectorization)

**Goal**: Use jax.vmap to automatically batch operations.

### Instructions:

1. Define a function apply_affine(vector, matrix, bias) that takes a single 1D vector, a 2D matrix, and a 1D bias. It should compute jnp.dot(matrix, vector) + bias.
2. You have a batch of vectors (a 2D array where each row is a vector), but a single matrix and a single bias that should be applied to each vector in the batch.
3. Use jax.vmap to create batched_apply_affine that efficiently applies apply_affine to each vector in the batch.
 - Hint: in_axes for jax.vmap should specify 0 for the batched vector argument, and None for matrix and bias as they are not batched (broadcasted). The out_axes should be 0 to indicate the output is batched along the first axis.
4. Test batched_apply_affine with sample data.

In [ ]:
# Instructions for Exercise 4
key_ex4_main, main_key = jax.random.split(main_key)
key_ex4_vec, key_ex4_mat, key_ex4_bias = jax.random.split(key_ex4_main, 3)

# 1. Define apply_affine for a single vector
def apply_affine(vector: jnp.ndarray, matrix: jnp.ndarray, bias: jnp.ndarray) -> jnp.ndarray:
    # DONE: Compute jnp.dot(matrix, vector) + bias
    result = jnp.dot(matrix, vector) + bias
    return result

# 2. Prepare data
batch_size = 4
input_features = 3
output_features = 2

# batch_of_vectors: (batch_size, input_features)
# single_matrix: (output_features, input_features)
# single_bias: (output_features,)
batch_of_vectors = jax.random.normal(key_ex4_vec, (batch_size, input_features))
single_matrix = jax.random.normal(key_ex4_mat, (output_features, input_features))
single_bias = jax.random.normal(key_ex4_bias, (output_features,))


# 3. Use jax.vmap to create batched_apply_affine
# DONE: Specify in_axes correctly: vector is batched, matrix and bias are not. out_axes should be 0.
# batched_apply_affine = None # Placeholder jax.vmap(apply_affine, in_axes=(..., ... , ...), out_axes=...)
batched_apply_affine = jax.vmap(
    apply_affine,
    in_axes=(0, None, None),
    out_axes=(0)
)

# 4. Test batched_apply_affine
# result_vmap = None # Placeholder batched_apply_affine(batch_of_vectors, single_matrix, single_bias)
result_vmap = batched_apply_affine(batch_of_vectors, single_matrix, single_bias)
print("Batch of vectors shape:", batch_of_vectors.shape)
print("Single matrix shape:", single_matrix.shape)
print("Single bias shape:", single_bias.shape)
if result_vmap is not None:
    print("Result using vmap shape:", result_vmap.shape) # Expected: (batch_size, output_features)

    # For comparison, a manual loop (less efficient):
    manual_results = []
    for i in range(batch_size):
        manual_results.append(apply_affine(batch_of_vectors[i], single_matrix, single_bias))
    result_manual_loop = jnp.stack(manual_results)
    assert jnp.allclose(result_vmap, result_manual_loop)
    print("vmap result matches manual loop result.")
else:
    print("result_vmap is None.")

Batch of vectors shape: (4, 3)
Single matrix shape: (2, 3)
Single bias shape: (2,)
Result using vmap shape: (4, 2)
vmap result matches manual loop result.


In [10]:
batch_of_vectors

Array([[ 0.13304706, -0.57423145,  1.267014  ],
       [-0.6409235 , -1.8416666 , -0.79874533],
       [ 1.1374704 , -0.14219557, -0.39751512],
       [-0.33728084, -0.7911733 ,  0.0659703 ]], dtype=float32)

In [11]:
single_matrix

Array([[-1.2056134 ,  1.4214323 , -0.50741124],
       [-0.72862333, -0.37455186,  1.0456928 ]], dtype=float32)

In [12]:
single_bias

Array([ 1.6305339 , -0.79477346], dtype=float32)

In [101]:
# @title Solution 4: `jax.vmap` (Automatic Vectorization)
key_ex4_sol_main, main_key = jax.random.split(main_key)
key_ex4_sol_vec, key_ex4_sol_mat, key_ex4_sol_bias = jax.random.split(key_ex4_sol_main, 3)

# 1. Define apply_affine for a single vector
def apply_affine_sol(vector: jnp.ndarray, matrix: jnp.ndarray, bias: jnp.ndarray) -> jnp.ndarray:
    return jnp.dot(matrix, vector) + bias

# 2. Prepare data
batch_size_sol = 4
input_features_sol = 3
output_features_sol = 2

batch_of_vectors_sol = jax.random.normal(key_ex4_sol_vec, (batch_size_sol, input_features_sol))
single_matrix_sol = jax.random.normal(key_ex4_sol_mat, (output_features_sol, input_features_sol))
single_bias_sol = jax.random.normal(key_ex4_sol_bias, (output_features_sol,))

# 3. Use jax.vmap to create batched_apply_affine
# Vector is batched along axis 0, matrix and bias are not batched (broadcasted).
# out_axes=0 means the output will also be batched along its first axis.
batched_apply_affine_sol = jax.vmap(apply_affine_sol, in_axes=(0, None, None), out_axes=0)

# 4. Test batched_apply_affine
result_vmap_sol = batched_apply_affine_sol(batch_of_vectors_sol, single_matrix_sol, single_bias_sol)
print("Batch of vectors shape:", batch_of_vectors_sol.shape)
print("Single matrix shape:", single_matrix_sol.shape)
print("Single bias shape:", single_bias_sol.shape)
print("Result using vmap shape:", result_vmap_sol.shape) # Expected: (batch_size, output_features)
assert result_vmap_sol.shape == (batch_size_sol, output_features_sol)

# For comparison, a manual loop (less efficient):
manual_results_sol = []
for i in range(batch_size_sol):
    manual_results_sol.append(apply_affine_sol(batch_of_vectors_sol[i], single_matrix_sol, single_bias_sol))
result_manual_loop_sol = jnp.stack(manual_results_sol)

assert jnp.allclose(result_vmap_sol, result_manual_loop_sol)
print("\nvmap result matches manual loop result, demonstrating correct vectorization.")

Batch of vectors shape: (4, 3)
Single matrix shape: (2, 3)
Single bias shape: (2,)
Result using vmap shape: (4, 2)

vmap result matches manual loop result, demonstrating correct vectorization.


#### Playing with `jax.vmap`

In [13]:
A, B, C, D = 2, 3, 4, 5
x = jnp.ones((A, B))
y = jnp.ones((B, C))
z = jnp.ones((C, D))

In [21]:
print(f"x, {x.shape}")
display(x)
print("="*30)

print(f"\ny, {y.shape}")
display(y)
print("="*30)

print(f"\nz, {z.shape}")
display(z)
print()

x, (2, 3)


Array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)


y, (3, 4)


Array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)


z, (4, 5)


Array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32)

In [23]:
def foo(tree_arg):
    x, (y, z) = tree_arg
    return jnp.dot(x, jnp.dot(y, z))

tree = (x, (y, z))
print(foo(tree))

[[12. 12. 12. 12. 12.]
 [12. 12. 12. 12. 12.]]


In [25]:
K = 6  # batch size
x = jnp.ones((K, A, B))  # batch axis in different locations
y = jnp.ones((B, K, C))
z = jnp.ones((C, D, K))

In [26]:
print(f"x, {x.shape}")
display(x)
print("="*30)

print(f"\ny, {y.shape}")
display(y)
print("="*30)

print(f"\nz, {z.shape}")
display(z)
print()

x, (6, 2, 3)


Array([[[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)


y, (3, 6, 4)


Array([[[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]], dtype=float32)


z, (4, 5, 6)


Array([[[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]]], dtype=float32)

In [ ]:
tree = (x, (y, z))

In [ ]:
#x: (6, 2, 3)
#y:        (3, 6, 4)
#z:               (4, 5, 6)

In [ ]:
vfoo = jax.vmap(
    foo, 
    in_axes=( # specifies which input array axes to map over.
        (
            0, 
            (1, 2)
        ),
    )
)

In [30]:
def dot(v1, v2):
    return jnp.vdot(v1, v2)

v1 = jnp.array([1., 1., 1.])
v2 = jnp.array([1., 2., -1.])

dot(v1, v2)

Array(2., dtype=float32)

In [70]:
rng_key = jax.random.PRNGKey(42)

vs = jax.random.normal(rng_key, shape=(20,3))

v1s = vs[:10, :]
v2s = vs[10:, :]

In [71]:
v1s.shape, v2s.shape

((10, 3), (10, 3))

In [ ]:
# A.1 Naive approach #1 --> wrong result & wrong shape/dimensions
dot(v1s, v2s)

Array(0.0978545, dtype=float32)

In [ ]:
# A.2 Naive approach #2 --> performance degradation and wrong type
[
    dot(v1s[i], v2s[i]) for i in range(v1s.shape[0])
]

## - Two caveats with this approach
## Caveat # 1 (insignificant)
#       You get a Python list of `Array`s, not a single `Array` with multidimensional array inside. Meaning you're gonna have to convert it because JAX does not accept Python list
## Caveat # 2 (significant)
#       performance degradation



[Array(-0.21668759, dtype=float32),
 Array(0.0147948, dtype=float32),
 Array(-0.7736949, dtype=float32),
 Array(-0.37052184, dtype=float32),
 Array(-0.19051453, dtype=float32),
 Array(-0.44745094, dtype=float32),
 Array(-1.2089032, dtype=float32),
 Array(1.1151277, dtype=float32),
 Array(0.02767059, dtype=float32),
 Array(2.1480343, dtype=float32)]

In [ ]:
# B. Manual vectorization
#   Change your input tensors to have an additional batch dimension
#   Then rewrite the calculation to use it.
# Con: this becomes error prone for more complex functions

def dot_vectorized(v1s, v2s):
    return jnp.einsum("ij,ij->i", v1s, v2s)

dot_vectorized(v1s, v2s)

Array([-0.21668759,  0.0147948 , -0.7736949 , -0.37052184, -0.19051453,
       -0.44745094, -1.2089032 ,  1.1151277 ,  0.02767059,  2.1480343 ],      dtype=float32)

In [41]:
# C. the convention in JAX is to use vmap:
dot_vmapped = jax.vmap(dot)
dot_vmapped(v1s, v2s)

Array([-0.21668759,  0.0147948 , -0.7736949 , -0.37052184, -0.19051453,
       -0.44745094, -1.2089032 ,  1.1151277 ,  0.02767059,  2.1480343 ],      dtype=float32)

In [45]:
dot_vectorized_jitted = jax.jit(dot_vectorized)
dot_vmapped_jitted = jax.jit(dot_vmapped)

In [42]:
# let's compare the approaches

%timeit [dot(v1s[i], v2s[i]).block_until_ready() for i in range(v1s.shape[0])]

3.84 ms ± 276 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
%timeit dot_vectorized(v1s, v2s).block_until_ready()

152 μs ± 1.76 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [48]:
%timeit dot_vmapped(v1s, v2s).block_until_ready()

322 μs ± 1.31 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%timeit dot_vectorized_jitted(v1s, v2s).block_until_ready()

42.2 μs ± 1.01 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%timeit dot_vmapped_jitted(v1s, v2s).block_until_ready()

41.5 μs ± 161 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [49]:
# Verdict:
# The naive approach is very slow & inefficient. 
# The manually vectorized function delivers the best perf
# The autom. vectorized fn is a bit sower yet still an order of magnitude faster than the naive approach
# After JIT-compiling both solutions, their speeds become similar
# 

In [50]:
# let's zoom into the internals and gen `jaxpr`s of these functions

dot_jaxpr = jax.make_jaxpr(dot)
dot_jaxpr

<function jax.make_jaxpr(dot)(v1, v2)>

In [ ]:
jax.make_jaxpr(dot)(
    jnp.array([1.,1., 1.]),
    jnp.array([1.,1., 1.])
)

{ lambda ; a:f32[3] b:f32[3]. let
    c:f32[] = dot_general[
      dimension_numbers=(([0], [0]), ([], []))
      preferred_element_type=float32
    ] a b
  in (c,) }

In [57]:
jax.make_jaxpr(dot_vectorized)(v1s, v2s)

{ lambda ; a:f32[10,3] b:f32[10,3]. let
    c:f32[10] = dot_general[
      dimension_numbers=(([1], [1]), ([0], [0]))
      preferred_element_type=float32
    ] a b
  in (c,) }

In [58]:
jax.make_jaxpr(dot_vmapped)(v1s, v2s)

{ lambda ; a:f32[10,3] b:f32[10,3]. let
    c:f32[10] = dot_general[
      dimension_numbers=(([1], [1]), ([0], [0]))
      preferred_element_type=float32
    ] a b
  in (c,) }

**Controlling array axes to map over using vmap's `in_axes`**

* if the `in_axes` param is an int, then the array axis specified by this number is used to map over all the fn args
* the `None` value means we do not map over this particular param
* the general rule is the `in_axes` structure should correspond to the structure of the associate inputs

In [59]:
dot_vmapped_as_before = jax.vmap(dot, in_axes=(0,0))

In [72]:
def scaled_dot(v1, v2, koeff):
    return koeff * jnp.vdot(v1, v2)

v1s_ = v1s
v2s_ = v2s.T
k = 1.0

v1s_.shape, v2s.shape

((10, 3), (10, 3))

In [73]:
scaled_dot(v1s_, v2s_, k)

Array(3.8348422, dtype=float32)

In [75]:
scaled_dot_batched = jax.vmap(
    scaled_dot, 
    in_axes=(   # iterate over the axis with index:
        0,      #   ...0 for the 1st param,
        1,      #   ...1 for the 2nd param.
        None    # but do not iterate over the third param, which contains just a scalar
    )
)

scaled_dot_batched(v1s_, v2s_, k)
# see  also scaled_dot_batched2() below

Array([-0.21668759,  0.0147948 , -0.7736949 , -0.37052184, -0.19051453,
       -0.44745094, -1.2089032 ,  1.1151277 ,  0.02767059,  2.1480343 ],      dtype=float32)

In [74]:
def scaled_dot2(data, koeff):
    return koeff * jnp.vdot(data["a"], data["b"])

scaled_dot_batched2 = jax.vmap(
    scaled_dot2,
    in_axes=(
        {"a": 0, "b": 1},
        None
    )
)

scaled_dot_batched2({"a": v1s_, "b": v2s_}, k)

Array([-0.21668759,  0.0147948 , -0.7736949 , -0.37052184, -0.19051453,
       -0.44745094, -1.2089032 ,  1.1151277 ,  0.02767059,  2.1480343 ],      dtype=float32)

**Controlling output array axes with vmap's `out_axes`**

In [87]:
def scale(v, koeff):
    return koeff * v

def scale_named_arg(v, koeff=1.0):
    return koeff * v

In [77]:
scale_batched = jax.vmap(
    scale, 
    in_axes=(0, None),
    out_axes=(1) # here we specify the batch dimension for the output
)

In [79]:
scale_batched(v1s, 1.0)

Array([[-0.02830462,  0.15354592, -1.4408789 ,  0.9101704 ,  1.4457862 ,
         0.9095945 , -1.4485087 , -1.179381  , -0.24111967, -0.5347689 ],
       [ 0.46713185, -0.12403282,  0.7558599 , -0.3844966 ,  1.0809066 ,
         0.55734617,  0.7641875 , -1.9389184 ,  1.2151275 ,  0.27067557],
       [ 0.29570296,  0.21692315,  0.52140963,  1.1398233 , -0.05629321,
         0.21905719, -0.24154697,  0.35626462, -1.3952221 ,  1.5401242 ]],      dtype=float32)

In [84]:
scale_batched_named_arg = jax.vmap(
    scale_named_arg, 
    in_axes=(0, None),
    out_axes=(1) # here we specify the batch dimension for the output
)

In [89]:
# scale_batched_named_arg(v1s, koeff=1.0)
## the above won't work
## to fix it you can  either:
##  - sol1: make a wrapper funtion that hides your named param, or
##  - sol2: you can broadcast the named param into an array with the required dimension to map over

In [ ]:
# sol1 demo (one way):

from functools import partial

scale_with_koef_2 = partial(scale_named_arg, koeff=2.0)
scale_batched_named_arg_with_koef_2 = jax.vmap(
    scale_with_koef_2,
    in_axes=(0),
    out_axes=(1) 
)

scale_batched_named_arg_with_koef_2(v1s)

Array([[-0.05660923,  0.30709183, -2.8817577 ,  1.8203408 ,  2.8915725 ,
         1.819189  , -2.8970175 , -2.358762  , -0.48223934, -1.0695378 ],
       [ 0.9342637 , -0.24806564,  1.5117198 , -0.7689932 ,  2.1618133 ,
         1.1146923 ,  1.528375  , -3.8778367 ,  2.430255  ,  0.54135114],
       [ 0.5914059 ,  0.4338463 ,  1.0428193 ,  2.2796466 , -0.11258642,
         0.43811437, -0.48309395,  0.71252924, -2.7904441 ,  3.0802484 ]],      dtype=float32)

In [97]:
# sol1 demo (decorator way, so you don't have to use intermediate fns):
from functools import partial

@partial(jax.vmap, in_axes=(0, None), out_axes=(1))
def scale_with_decorators(v, koeff):
    return koeff * v

scale_with_decorators(v1s, 2.0)

Array([[-0.05660923,  0.30709183, -2.8817577 ,  1.8203408 ,  2.8915725 ,
         1.819189  , -2.8970175 , -2.358762  , -0.48223934, -1.0695378 ],
       [ 0.9342637 , -0.24806564,  1.5117198 , -0.7689932 ,  2.1618133 ,
         1.1146923 ,  1.528375  , -3.8778367 ,  2.430255  ,  0.54135114],
       [ 0.5914059 ,  0.4338463 ,  1.0428193 ,  2.2796466 , -0.11258642,
         0.43811437, -0.48309395,  0.71252924, -2.7904441 ,  3.0802484 ]],      dtype=float32)

In [94]:
# sol2 aka broadcast demo:

scale_batched_named_arg_with_broadcast = jax.vmap(
    scale,
    in_axes=(0),
    out_axes=(1) 
)

scale_batched_named_arg_with_broadcast(
    v1s,
    koeff=jnp.broadcast_to(
        2.0,
        (v1s.shape[0],)
    )
)

Array([[-0.05660923,  0.30709183, -2.8817577 ,  1.8203408 ,  2.8915725 ,
         1.819189  , -2.8970175 , -2.358762  , -0.48223934, -1.0695378 ],
       [ 0.9342637 , -0.24806564,  1.5117198 , -0.7689932 ,  2.1618133 ,
         1.1146923 ,  1.528375  , -3.8778367 ,  2.430255  ,  0.54135114],
       [ 0.5914059 ,  0.4338463 ,  1.0428193 ,  2.2796466 , -0.11258642,
         0.43811437, -0.48309395,  0.71252924, -2.7904441 ,  3.0802484 ]],      dtype=float32)

## Exercise 5: Flax NNX - Defining a Model

**Goal**: Learn to define a simple neural network model using Flax NNX.

### Instructions:

1. Define a Flax NNX model class SimpleNNXModel that inherits from nnx.Module.
2. In its __init__, define one nnx.Linear layer. The layer should take din (input features) and dout (output features) as arguments. Remember to pass the rngs argument to nnx.Linear for parameter initialization (e.g., rngs=rngs).
3. Implement the __call__ method (the forward pass) which takes an input x and passes it through the linear layer.
4. Instantiate your SimpleNNXModel. You'll need to create an nnx.Rngs object using a JAX PRNG key (e.g., nnx.Rngs(params=jax.random.key(seed))). The key name params is conventional for nnx.Linear.
5. Test your model instance with a dummy input batch. Print the output and the model's state (parameters) using nnx.display().

In [ ]:
# Instructions for Exercise 5
key_ex5_model_init, main_key = jax.random.split(main_key)

# 1. & 2. & 3. Define the SimpleNNXModel
class SimpleNNXModel(nnx.Module):
    def __init__(self, din: int, dout: int, *, rngs: nnx.Rngs):
        # TODO: Define an nnx.Linear layer named 'dense_layer'
        # self.dense_layer = nnx.Linear(...)
        self.some_attribute = None # Placeholder, remove later
        pass # Remove this placeholder if class is not empty

    def __call__(self, x: jnp.ndarray) -> jnp.ndarray:
        # TODO: Pass input x through the dense_layer
        # return self.dense_layer(x)
        return x # Placeholder

# 4. Instantiate the model
model_din = 3
model_dout = 2
# TODO: Create nnx.Rngs for parameter initialization. Use 'params' as the key name.
model_rngs = None # Placeholder nnx.Rngs(params=key_ex5_model_init)
my_model = None # Placeholder SimpleNNXModel(din=model_din, dout=model_dout, rngs=model_rngs)

# 5. Test with dummy data
dummy_batch_size = 4
dummy_input_ex5 = jnp.ones((dummy_batch_size, model_din))

model_output = None # Placeholder
if my_model is not None:
    model_output = my_model(dummy_input_ex5)
    print(f"Model output shape: {model_output.shape}")
    print(f"Model output:\n{model_output}")

    model_state = my_model.get_state()
    print(f"\nModel state (parameters, etc.):")
    pprint.pprint(model_state)
else:
    print("my_model is None.")

## Exercise 6: Optax & Flax NNX - Creating an Optimizer

**Goal**: Set up an Optax optimizer and wrap it with nnx.Optimizer for use with a Flax NNX model.

### Instructions:
1. Use the SimpleNNXModel_Sol class and an instance my_model_sol from the previous exercise's solution. (If running standalone, re-instantiate it).
2. Create an Optax optimizer, for example, optax.adam with a learning rate of 0.001.
3. Create an nnx.Optimizer instance. This wrapper links the Optax optimizer with your Flax NNX model (my_model_sol).
4. Print the nnx.Optimizer instance and its state attribute to see the initialized optimizer state (e.g., Adam's momentum terms).

In [ ]:
# Instructions for Exercise 6

# 1. Assume my_model_sol is available from Exercise 5 solution
# (If running standalone, re-instantiate it)
if 'my_model_sol' not in globals():
    print("Re-initializing model from Ex5 solution for Ex6.")
    key_ex6_model_init, main_key = jax.random.split(main_key)
    _model_din_ex6 = 3
    _model_dout_ex6 = 2
    _model_rngs_ex6 = nnx.Rngs(params=key_ex6_model_init)
    # Use solution class name if defined, otherwise student's class name
    _ModelClass = SimpleNNXModel_Sol if 'SimpleNNXModel_Sol' in globals() else SimpleNNXModel
    model_for_opt = _ModelClass(din=_model_din_ex6, dout=_model_dout_ex6, rngs=_model_rngs_ex6)
    print("Model for optimizer created.")
else:
    model_for_opt = my_model_sol # Use the one from previous solution
    print("Using model 'my_model_sol' from previous exercise for 'model_for_opt'.")


# 2. Create an Optax optimizer
learning_rate = 0.001
# TODO: Create an optax.adam optimizer transform
optax_tx = None # Placeholder optax.adam(...)

# 3. Create an nnx.Optimizer wrapper
# TODO: Wrap the model (model_for_opt) and the optax transform (optax_tx)
# The `wrt` argument is now required to specify what to differentiate with respect to.
nnx_optimizer = None # Placeholder nnx.Optimizer(...)

# 4. Print the optimizer and its state
print("\nFlax NNX Optimizer wrapper:")
nnx.display(nnx_optimizer)

print("\nInitial Optimizer State (Optax state, e.g., Adam's momentum):")
if nnx_optimizer is not None and hasattr(nnx_optimizer, 'opt_state'):
   pprint.pprint(nnx_optimizer.state)
   # if hasattr(nnx_optimizer, 'opt_state'):
   #     adam_state = nnx_optimizer.opt_state
   #     assert len(adam_state) > 0 and hasattr(adam_state[0], 'count')
   #     print("\nOptimizer state structure looks plausible for Adam.")
else:
   print("nnx_optimizer or its state is None or not structured as expected.")

## Exercise 7: Training Step with Flax NNX and Optax

**Goal**: Implement a complete JIT-compiled training step for a Flax NNX model using Optax.

### Instructions:

1. You'll need:
 - An instance of your model class (e.g., my_model_sol from Ex 5/6 solution).
 - An instance of nnx.Optimizer (e.g., nnx_optimizer_sol from Ex 6 solution).
2. Define a train_step function that is decorated with @nnx.jit. This function should take the model, optimizer, input x_batch, and target y_batch as arguments.
3. Inside train_step:
 - Define an inner loss_fn_for_grad. This function must take the model as its first argument. Inside, it computes the model's predictions for x_batch and then calculates the mean squared error (MSE) against y_batch.
 - Use nnx.value_and_grad(loss_fn_for_grad)(model_arg) to compute both the loss value and the gradients with respect to the model passed to loss_fn_for_grad. (model_arg is the model instance passed into train_step).
 - Update the model's parameters (and the optimizer's state) using optimizer_arg.update(model_arg, grads). The update method takes the model and gradients, and updates the model's state in-place.
 - Return the computed loss_value.
4. Create dummy x_batch and y_batch data.
5. Call your train_step function. Print the returned loss.
6. (Optional) Verify that the model's parameters have changed after the train_step by comparing a parameter value before and after the call.

In [ ]:
# Instructions for Exercise 7
key_ex7_main, main_key = jax.random.split(main_key)
key_ex7_x, key_ex7_y = jax.random.split(key_ex7_main, 2)

# 1. Use model and optimizer from previous exercises' solutions
# Ensure my_model_sol and nnx_optimizer_sol are available
if 'my_model_sol' not in globals() or 'nnx_optimizer_sol' not in globals():
    print("Re-initializing model and optimizer from Ex5/Ex6 solutions for Ex7.")
    key_ex7_model_fallback, main_key = jax.random.split(main_key)
    _model_din_ex7 = 3
    _model_dout_ex7 = 2
    _model_rngs_ex7 = nnx.Rngs(params=key_ex7_model_fallback)
    # Ensure SimpleNNXModel_Sol is used
    my_model_ex7 = SimpleNNXModel_Sol(din=_model_din_ex7, dout=_model_dout_ex7, rngs=_model_rngs_ex7)
    _optax_tx_ex7 = optax.adam(learning_rate=0.001)
    nnx_optimizer_ex7 = nnx.Optimizer(my_model_ex7, _optax_tx_ex7)
    print("Model and optimizer re-created for Ex7.")
else:
    my_model_ex7 = my_model_sol
    nnx_optimizer_ex7 = nnx_optimizer_sol
    print("Using 'my_model_sol' and 'nnx_optimizer_sol' for 'my_model_ex7' and 'nnx_optimizer_ex7'.")


# 2. & 3. Define the train_step function
# TODO: Decorate with @nnx.jit
# def train_step(model_arg: nnx.Module, optimizer_arg: nnx.Optimizer, # Type hint with base nnx.Module
#                x_batch: jnp.ndarray, y_batch: jnp.ndarray) -> jnp.ndarray:

    # TODO: Define inner loss_fn_for_grad(current_model_state_for_grad_fn)
    # def loss_fn_for_grad(model_in_grad_fn: nnx.Module): # Type hint with base nnx.Module
        # y_pred = model_in_grad_fn(x_batch)
        # loss = jnp.mean((y_pred - y_batch)**2)
        # return loss
    #    return jnp.array(0.0) # Placeholder

    # TODO: Compute loss value and gradients using nnx.value_and_grad
    # loss_value, grads = nnx.value_and_grad(loss_fn_for_grad)(model_arg) # Pass model_arg

    # TODO: Update the optimizer (which updates the model_arg in-place)
    # optimizer_arg.update(model_arg, grads)

    # return loss_value
#    return jnp.array(0.0) # Placeholder defined train_step function

# For the student to define:
# Make sure the function signature is correct for nnx.jit
@nnx.jit
def train_step(model_arg: nnx.Module, optimizer_arg: nnx.Optimizer,
               x_batch: jnp.ndarray, y_batch: jnp.ndarray) -> jnp.ndarray:
    # Placeholder implementation for student
    def loss_fn_for_grad(model_in_grad_fn: nnx.Module):
        # y_pred = model_in_grad_fn(x_batch)
        # loss = jnp.mean((y_pred - y_batch)**2)
        # return loss
        return jnp.array(0.0) # Student TODO: replace this

    # loss_value, grads = nnx.value_and_grad(loss_fn_for_grad)(model_arg)
    # optimizer_arg.update(grads)
    # return loss_value
    return jnp.array(-1.0) # Student TODO: replace this


# 4. Create dummy data
batch_s = 8
# Access features_in and features_out carefully
_din_from_model_ex7 = my_model_ex7.dense_layer.in_features if hasattr(my_model_ex7, 'dense_layer') else 3
_dout_from_model_ex7 = my_model_ex7.dense_layer.out_features if hasattr(my_model_ex7, 'dense_layer') else 2

x_batch_data = jax.random.normal(key_ex7_x, (batch_s, _din_from_model_ex7))
y_batch_data = jax.random.normal(key_ex7_y, (batch_s, _dout_from_model_ex7))

# Optional: Store initial param value for comparison
initial_kernel_val = None
if hasattr(my_model_ex7, 'get_state'):
    _current_model_state_ex7 = my_model_ex7.get_state()
    if 'dense_layer' in _current_model_state_ex7:
       initial_kernel_val = _current_model_state_ex7['dense_layer']['kernel'].value[0,0].copy()
print(f"Initial kernel value (sample): {initial_kernel_val}")

# 5. Call the train_step
# loss_after_step = train_step(my_model_ex7, nnx_optimizer_ex7, x_batch_data, y_batch_data) # Student will uncomment
loss_after_step = jnp.array(-1.0) # Placeholder until student implements train_step
if train_step(my_model_ex7, nnx_optimizer_ex7, x_batch_data, y_batch_data).item() != -1.0: # Check if student implemented
    loss_after_step = train_step(my_model_ex7, nnx_optimizer_ex7, x_batch_data, y_batch_data)
    print(f"Loss after one training step: {loss_after_step}")
else:
    print("Student needs to implement `train_step` function.")


# # 6. Optional: Verify parameter change
# updated_kernel_val_sol = None
# _, updated_model_state_sol = nnx.split(my_model_sol_ex7) # Get state again after update
# if 'dense_layer' in updated_model_state_sol:
#   updated_kernel_val_sol = updated_model_state_sol['dense_layer']['kernel'].value[0,0]
#   print(f"Updated kernel value (sample): {updated_kernel_val_sol}")

# if initial_kernel_val_sol is not None and updated_kernel_val_sol is not None:
#     assert not jnp.allclose(initial_kernel_val_sol, updated_kernel_val_sol), "Kernel parameter did not change!"
#     print("Kernel parameter changed as expected after the training step.")
# else:
#     print("Could not verify kernel change (initial or updated value was None).")

## Exercise 8: Orbax - Saving and Restoring Checkpoints

**Goal**: Learn to use Orbax to save and restore JAX PyTrees, specifically Flax NNX model states and Optax optimizer states.

### Instructions:
1. You'll need your model (e.g., my_model_sol_ex7) and optimizer (e.g., nnx_optimizer_sol_ex7) from the previous exercise's solution.
2. Define a checkpoint directory (e.g., /tmp/my_nnx_checkpoint/).
3. Create an Orbax CheckpointManagerOptions and then a CheckpointManager.
4. Bundle the states you want to save into a dictionary. For NNX, this is my_model_sol_ex7.get_state() for the model, and nnx_optimizer_sol_ex7.state for the optimizer's internal state. Also include a training step counter.
5. Use checkpoint_manager.save() with ocp.args.StandardSave() to save the bundled state. Call checkpoint_manager.wait_until_finished() to ensure saving completes.
6. To restore:
 - Create new instances of your model (restored_model) and Optax transform (restored_optax_tx). The new model should have a different PRNG key for its initial parameters to demonstrate that restoration works.
 - Use checkpoint_manager.restore() with ocp.args.StandardRestore() to load the bundled state.
 - Apply the loaded model state to restored_model using restored_model.update_state(loaded_bundle['model']).
 - Create a new nnx.Optimizer (restored_optimizer) associating restored_model and restored_optax_tx.
 - Assign the loaded optimizer state to the new optimizer: restored_optimizer.state = loaded_bundle['optimizer'].
7. Verify that a parameter from restored_model matches the corresponding parameter from the original my_model_sol_ex7 (before saving, or from the saved state). Also, compare optimizer states if possible.
8. Clean up the checkpoint directory.

In [ ]:
# Instructions for Exercise 8
# import orbax.checkpoint as ocp # Already imported
# import os, shutil # Already imported

# 1. Use model and optimizer from previous exercise solution
if 'my_model_sol_ex7' not in globals() or 'nnx_optimizer_sol_ex7' not in globals():
    print("Re-initializing model and optimizer from Ex7 solution for Ex8.")
    key_ex8_model_fallback, main_key = jax.random.split(main_key)
    _model_din_ex8 = 3
    _model_dout_ex8 = 2
    _model_rngs_ex8 = nnx.Rngs(params=key_ex8_model_fallback)
    _ModelClassEx8 = SimpleNNXModel_Sol if 'SimpleNNXModel_Sol' in globals() else SimpleNNXModel
    model_to_save = _ModelClassEx8(din=_model_din_ex8, dout=_model_dout_ex8, rngs=_model_rngs_ex8)
    _optax_tx_ex8 = optax.adam(learning_rate=0.001)
    optimizer_to_save = nnx.Optimizer(model_to_save, _optax_tx_ex8)
    print("Model and optimizer re-created for Ex8.")
else:
    model_to_save = my_model_sol_ex7
    optimizer_to_save = nnx_optimizer_sol_ex7
    print("Using model and optimizer from Ex7 solution for Ex8.")

# 2. Define checkpoint directory
# TODO: Define checkpoint_dir
checkpoint_dir = None # Placeholder e.g., "/tmp/my_nnx_checkpoint_exercise/"
# if checkpoint_dir and os.path.exists(checkpoint_dir):
#    shutil.rmtree(checkpoint_dir) # Clean up previous runs for safety
# if checkpoint_dir:
#    os.makedirs(checkpoint_dir, exist_ok=True)


# 3. Create Orbax CheckpointManager
# TODO: Create options and manager
# options = ocp.CheckpointManagerOptions(...)
# mngr = ocp.CheckpointManager(...)
options = None
mngr = None

# 4. Bundle states
# current_step = 100 # Example step
# TODO: Get model_state and optimizer_state
# model_state_to_save = nnx.split(model_to_save)
# The optimizer state is now accessed via the .state attribute.
# opt_state_to_save = optimizer_to_save.state
# save_bundle = {
#     'model': model_state_to_save,
#     'optimizer': opt_state_to_save,
#     'step': current_step
# }
save_bundle = None

# 5. Save the checkpoint
# if mngr and save_bundle:
#   TODO: Save checkpoint
#   mngr.save(...)
#   mngr.wait_until_finished()
#   print(f"Checkpoint saved at step {current_step} to {checkpoint_dir}")
# else:
#   print("Checkpoint manager or save_bundle not initialized.")

# --- Restoration ---
# 6.a Create new model and Optax transform (for restoration)
# key_ex8_restore_model, main_key = jax.random.split(main_key)
# din_restore = model_to_save.dense_layer.in_features if hasattr(model_to_save, 'dense_layer') else 3
# dout_restore = model_to_save.dense_layer.out_features if hasattr(model_to_save, 'dense_layer') else 2
# _ModelClassRestore = SimpleNNXModel_Sol if 'SimpleNNXModel_Sol' in globals() else SimpleNNXModel
# restored_model = _ModelClassRestore(
#     din=din_restore, dout=dout_restore,
#     rngs=nnx.Rngs(params=key_ex8_restore_model) # New key for different initial params
# )
# restored_optax_tx = optax.adam(learning_rate=0.001) # Same Optax config
restored_model = None
restored_optax_tx = None

# 6.b Restore the checkpoint
# loaded_bundle = None
# if mngr:
#   TODO: Restore checkpoint
#   latest_step = mngr.latest_step()
#   if latest_step is not None:
#       loaded_bundle = mngr.restore(...)
#       print(f"Checkpoint restored from step {latest_step}")
#   else:
#       print("No checkpoint found to restore.")
# else:
#   print("Checkpoint manager not initialized for restore.")

# 6.c Apply loaded states
# if loaded_bundle and restored_model:
#   TODO: Update restored_model state
#   nnx.update(restored_model, ...)
#   print("Restored model state applied.")

    # TODO: Create new nnx.Optimizer and assign its state
#   restored_optimizer = nnx.Optimizer(...)
#   restored_optimizer.state = ...
#   print("Restored optimizer state applied.")
# else:
#   print("Loaded_bundle or restored_model is None, cannot apply states.")
restored_optimizer = None

# 7. Verify restoration
# original_kernel_sol = save_bundle_sol['model']['dense_layer']['kernel']
# _, restored_model_state = nnx.split(restored_model_sol)
# kernel_after_restore_sol = restored_model_state['dense_layer']['kernel']
# assert jnp.array_equal(original_kernel_sol.value, kernel_after_restore_sol.value), \
#        "Model kernel parameters differ after restoration!"
# print("\nModel parameters successfully restored and verified (kernel match).")

# # Verify optimizer state (e.g., Adam's 'mu' for a specific parameter)
# original_opt_state_adam_mu_kernel = save_bundle_sol['optimizer'][0].mu['dense_layer']['kernel'].value
# restored_opt_state_adam_mu_kernel = restored_optimizer_sol.state[0].mu['dense_layer']['kernel'].value
# assert jnp.array_equal(original_opt_state_adam_mu_kernel, restored_opt_state_adam_mu_kernel), \
#                        "Optimizer Adam mu for kernel differs!"
# print("Optimizer state (sample mu) successfully restored and verified.")


# 8. Clean up
# if mngr:
#   mngr.close()
# if checkpoint_dir and os.path.exists(checkpoint_dir):
#   shutil.rmtree(checkpoint_dir)
#   print(f"Cleaned up checkpoint directory: {checkpoint_dir}")

## Conclusion

### Congratulations on completing the JAX AI Stack exercises!

You've now had a hands-on introduction to:

- Core JAX: jax.numpy, functional programming, jax.jit, jax.grad, jax.vmap.
- Flax NNX: Defining and instantiating Pythonic neural network models.
- Optax: Creating and using composable optimizers with Flax NNX.
- Training Loop: Implementing an end-to-end training step in Flax NNX.
- Orbax: Saving and restoring model and optimizer states.

This forms a strong foundation for developing high-performance machine learning models with the JAX ecosystem.

For further learning, refer to the official documentation:
- JAX AI Stack: https://jaxstack.ai
- JAX: https://jax.dev
- Flax NNX: https://flax.readthedocs.io
- Optax: https://optax.readthedocs.io
- Orbax: https://orbax.readthedocs.io

Don't forget to provide feedback on the training session:
https://goo.gle/jax-training-feedback